In [1]:
import pandas as pd
import re

In [2]:
# --- Day 7: Handy Haversacks ---
# You land at the regional airport in time for your next flight. 
# In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

# Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

# For example, consider the following rules:

# light red bags contain 1 bright white bag, 2 muted yellow bags.
# dark orange bags contain 3 bright white bags, 4 muted yellow bags.
# bright white bags contain 1 shiny gold bag.
# muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
# shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
# dark olive bags contain 3 faded blue bags, 4 dotted black bags.
# vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
# faded blue bags contain no other bags.
# dotted black bags contain no other bags.
# These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

# You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

# In the above rules, the following options would be available to you:

# A bright white bag, which can hold your shiny gold bag directly.
# A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
# A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
# A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
# So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

# How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [3]:
df = pd.read_excel('bags.xlsx')
df.head()

,bags
0,striped white bags contain 4 drab silver bags.
1,drab silver bags contain no other bags.
2,pale plum bags contain 1 dark black bag.
3,"muted gold bags contain 1 wavy red bag, 3 mirr..."
4,"muted teal bags contain 2 pale beige bags, 5 c..."


In [4]:
value = 'striped white bags contain 4 drab silver bags.'

value = re.findall(r'(.*)bags contain(.*)\.',value)

bags = value[0][0]
contents = value[0][1]

print(bags,contents)

striped white   4 drab silver bags


In [5]:
def get_bag_contents(value):
    
    value = re.findall(r'(.*)bags contain(.*)\.',value)

    return value[0][1]

In [6]:
def get_bags(value):
    
    value = re.findall(r'(.*)bags contain(.*)\.',value)

    return value[0][0].strip()

In [7]:
df['contents'] = df.bags.apply(get_bag_contents)
df['bags'] = df.bags.apply(get_bags)

In [8]:
df.head()

,bags,contents
0,striped white,4 drab silver bags
1,drab silver,no other bags
2,pale plum,1 dark black bag
3,muted gold,"1 wavy red bag, 3 mirrored violet bags, 5 bri..."
4,muted teal,"2 pale beige bags, 5 clear beige bags, 2 dott..."


In [9]:
len(df.bags)

594

In [10]:
total_bags = []
small_bags = ['shiny gold']
big_bags = []

getting_bags = True

while getting_bags:

    # get list of bags that contain at least one bag in small_bags (bigger_bags)
    for bag in small_bags:

        # add column 'contains {bag}' to dataframe 
        df[f'contains {bag}'] = df.contents.str.contains(bag)

        # create a dataframe showing only rows with a value of true for 'contains {bag}'
        df_temp = df[df[f'contains {bag}'] == True]

        # create a list of bags using the bags column of the dataframe
        li = list(df_temp.bags)

        # add list of bags to bigger_bags
        big_bags.extend(li)

    # if big_bags id empty break loop
    if len(big_bags) == 0:

        break

    # add bags list to master_list
    total_bags.extend(big_bags)

    # set bags list to equal bigger_bags list
    small_bags = big_bags

    # set bigger_bags to empty
    big_bags = []

# print number of items in total bags
print(len(set(total_bags)))

103


In [11]:
# --- Part Two ---
# It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

# Consider again your shiny gold bag and the rules from the above example:

# faded blue bags contain 0 other bags.
# dotted black bags contain 0 other bags.
# vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
# dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.
# So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

# Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

# Here's another example:

# shiny gold bags contain 2 dark red bags.
# dark red bags contain 2 dark orange bags.
# dark orange bags contain 2 dark yellow bags.
# dark yellow bags contain 2 dark green bags.
# dark green bags contain 2 dark blue bags.
# dark blue bags contain 2 dark violet bags.
# dark violet bags contain no other bags.
# In this example, a single shiny gold bag must contain 126 other bags.

# How many individual bags are required inside your single shiny gold bag?

In [12]:
df = df[['bags','contents']]
df.head()

,bags,contents
0,striped white,4 drab silver bags
1,drab silver,no other bags
2,pale plum,1 dark black bag
3,muted gold,"1 wavy red bag, 3 mirrored violet bags, 5 bri..."
4,muted teal,"2 pale beige bags, 5 clear beige bags, 2 dott..."


In [13]:
df.head(1)

,bags,contents
0,striped white,4 drab silver bags


In [14]:
def multiply_text(value):
    
    s =  ''
    value_list = value.split(',')

    for v in value_list:

        mod = re.findall(r'(\d)\s(\w*)\s(\w*)\sbag',v)
        
        if len(mod) > 0:
            
            mod = mod[0]

            number = int(mod[0])

            disc = mod[1]

            bag = mod[2]

            phrase = (disc + bag + ' ') * number

            s += phrase

    return s

In [15]:
df = df[['bags','contents']]

df['contents'] = df.contents.apply(multiply_text)
df['bags'] = df.bags.apply(lambda value : re.sub('\s','', value) )

df

,bags,contents
0,stripedwhite,drabsilver drabsilver drabsilver drabsilver
1,drabsilver,
2,paleplum,darkblack
3,mutedgold,wavyred mirroredviolet mirroredviolet mirrored...
4,mutedteal,palebeige palebeige clearbeige clearbeige clea...
...,...,...
589,vibrantorange,
590,stripedsilver,clearorange clearorange clearorange clearorang...
591,clearcyan,mutedgray mutedgray mutedgray mutedgray mutedg...
592,lightblack,stripedyellow


In [16]:
total_bags = []
big_bags = ['shinygold']
small_bags = []

getting_bags = True

while getting_bags:

    # get list of bags that contained in big_bags (small_bags)
    for bag in big_bags:

        # get list of bags contained by big_bags 
        df_temp = df[['contents']][df.bags == f'{bag}']

        if list(df_temp.contents) != []:
        
            li = list(df_temp.contents)[0].strip().split(' ')

            # add list to small bags
            small_bags.extend(li)

    #if small_bags is empty break loop
    if len(small_bags) == 0:

        break

    # add small_bags list to master_list
    total_bags.extend(small_bags)

    # set big_bags equal to small_bags
    big_bags = small_bags

    # set small_bags to empty
    small_bags = []
    
    
# remove all '' from list

while("" in total_bags):
    
    total_bags.remove('')

print(total_bags)

# print number of items in total bags
print(len(total_bags))

['vibrantblue', 'vibrantblue', 'vibrantblue', 'plaidblue', 'plaidblue', 'plaidblue', 'plaidblue', 'plaidblue', 'darkred', 'darkred', 'dullgreen', 'wavyturquoise', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'wavyturquoise', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'wavyturquoise', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'dimsalmon', 'dulllavender', 'dulllavender', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'lightindigo', 'dulllavender', 'dulllavender', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'lightindigo', 'dulllavender', 'dulllavender', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'lightindigo', 'dulllavender', 'dulllavender', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'lightindigo', 'dulllavender', 'dulllavender', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'wavymagenta', 'lightindigo', 'wavybronze', 'wavybronze', 'wavybron